In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [6]:
scales = [5]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    # FIXME: should be replaced by `s.where` when it's available. Now the result may be incorrect
    filter = (x > 20)
    filtered_x = s.where(filter, x)
    return s.mean(filtered_x)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:171: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



 <------------- Numerical Fidelity Report (input_scale: 5, param_scale: 5, scale_input_multiplier: 10) ------------->

+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error   | median_error | max_error | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+--------------+--------------+-----------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.001045227 | -0.002090454 | 0         | -0.002090454 | 0.001045227    | 0.002090454      | 0.002090454   | 0             | 0.0000021849992    | -0.00001923436     | 0.00001923436          |
+--------------+--------------+-----------+--------------+----------------+------------------+---------

==== Generate & Calibrate Setting ====
scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[-1178,5114],"logrows":13,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":3936,"total_assignments":911,"total_const_size":205,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5],"module_sizes":{"kzg":[],"poseidon":[3936,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709616804587}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.7909603118896484 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 54.34375
==== Generating Proof ====
proof:  {'instances': [['cde4916bd205742cffb14e51076823185ef1d3df4377d71dddd8ab4fd1be1002', '0100000000000000000000000000000000000000000000000000000000000000', 'cb06000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x1d3c469461328721b78dfface5e21f9a964fc21e71a76ed225eec043019ffcd5060d8c1f2f29ecbf3825f6ffcaf2c53e2392eed5e5f0717ec89edbed3df61c8b043305a05596b3f1906898aa9b0868ddb9b9a3366d8ce127b5d47572d2616531027c3bff5e4972b891a179b94c173c4183c2fd7928be724b7082f4b5c4c5be8426c4429c418ff25bfdc75e396e171fe057d48fbfa2c1d3247a856cf1ff872bef00e25ccb3e2d18ea24a04fec8c295702527c1f2dee8c63481ca83b1e818420c030474a1e06b1d06dd9f90a84996f5b8bf3e3990df23acd7e55a0c95f5d5c4ceb006aab21a3ba10e47823be8899d055e9d250579d28ddc80d81948fb8d6861fb11fc31aa933bcf066a258b28923c2bbeeffdbcf68a6e54db15927dacc495923a32cc24792bb3

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [54.34375]
